In [ ]:
#%matplotlib notebook
#%matplotlib notebook

import numpy as np
import cv2
import matplotlib.pyplot as mpp
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage as ndi
from ipywidgets import interact, interactive , fixed
from ipywidgets import VBox, HBox
import ipywidgets as widgets
from IPython.display import display
from scipy import stats
import pandas as pd
from sklearn.metrics import r2_score
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

# <font color = royalblue> $ \mathbb{Steps \ to \ implement \ the \ Watershed \ algorithm} $
<font color = white> $ 1:  \ \ \  \boxed{ \mathbb{Blur \ the \ image \ with \ smooth \ filter}} $ <br> 
<font color = white> $ 2:  \ \ \  \boxed{ \mathbb{Find  \ local \ maximas }} $ <br>
<font color = white> $ 3:  \ \ \  \boxed{ \mathbb{Draw \ the \ points \ in \ the \ image}} $ <br>
<font color = white>$  4: \ \ \  \boxed{ \mathbb{Contour \ and \ label \ the \ points }} $ <br>
<font color = white> $ 5:  \ \ \  \boxed{ \mathbb{Redraw \ by \ weighting \ the \ size \ of \ each \ point \ individually }} $ <br>
<font color = white> $ 6: \ \ \ \  \boxed{ \mathbb{Discover \ the \ unkown \ area \ and \ dilate \ with \ morphological \ operation}} $ <br>
<font color = white> $ 7: \ \ \ \  \boxed{ \mathbb{Connect \ components \ by \ neighborhood}} $ <br>
<font color = white> $ 8: \ \ \ \  \boxed{ \mathbb{Apply \ the \ watershed \ segmentation}} $ <br>
<font color = white> $ 9: \ \ \   \boxed{ \mathbb{Visualize \ the \ dam \ lines \ in \ the \ DEM \ of \ the \ crop \ height}} $
<font color = white> $ 10: \ \ \   \boxed{ \mathbb{Compare \ the \ regressions \ of \ the \ field \ data \ and \ drone \ data  }} $
    

### <font color = royalblue> $ \mathbb{Read \ the \ raster \ DEM \ in \ uint8 \ format \ (image)} $

In [ ]:
t = raw_input("What digital elevation model apply the sequence of algorithms ?")
img = cv2.imread(t,0)
print img.shape
print img.dtype
# C:\Users\G1745 IRON\Desktop\Forestry_Area\Watershed\Block_IZ1\Plantio/Plantio_dem_uint8.tif

### <font color = royalblue> $ \mathbb{Read \ the \ raster \ RGB \ in \ uint8 \ format} $

In [ ]:
s = raw_input('What is the RGB image to evaluate the result ?')
bgr_image = cv2.imread(s,1)
rgb_image = cv2.cvtColor(bgr_image,cv2.COLOR_BGR2RGB)
print rgb_image.shape
print rgb_image.dtype
# C:\Users\G1745 IRON\Desktop\Forestry_Area\Watershed\Block_IZ1\Plantio/Plantio_RGB.tif

### <font color = royalblue> $ \mathbb{Read \ the \ raster \ DEM \ in \ float64 \ format \ (array)} $

In [ ]:
height_plantio = np.load('plantio.npy');
height_plantio = np.flipud(height_plantio);
print height_plantio.shape
print height_plantio.dtype

### <font color =royalblue> $ \mathbb{Read \ the \ field \ data \ measurement \ .csv \ format} $

In [ ]:
t = raw_input('what data do you want import ?')
mydata = pd.read_csv(t,sep=',')
df_whole = pd.DataFrame(mydata)
df_whole['Crown_area_IZ1'] *= 10000
display(df_whole.head(2))
df_whole.shape
df_whole.dtypes
#df_whole.iloc[:,1] = Height
#df_whole.iloc[:,2] = Crown Area
# C:\Users\G1745 IRON\Desktop\Forestry_Area\Watershed\Block_IZ1\Plantio/plantio.csv

In [ ]:
mpp.rcParams["font.family"] = "Times New Roman"
mpp.rcParams.update({'font.size': 8})
mpp.rcParams["font.weight"] = "normal" 
mpp.rcParams["axes.labelweight"] = "normal"
mpp.rcParams['axes.titlesize'] = 8
mpp.rcParams['axes.titleweight'] = "normal"

### <font color = gold> $ 1. \ \ \ \mathbb{Define \ the \ interactive \ widgets} $

In [ ]:
min_distance = widgets.IntSlider(value=20,min=1,max=100,step=1.,description='',readout_format='d', continuous_update = False)
min_distance_txt = widgets.IntText(value=20,min=1,max=100,step=1.,description='',readout_format='d', continuous_update = False)
label_01 = widgets.Label(value='Minimum search distance')
l1 = widgets.jslink((min_distance,'value'),(min_distance_txt,'value'))

threshold_01 = widgets.IntSlider(value=50,min=0,max=255,step=1.,description='',readout_format='d', continuous_update = False)
threshold_01_txt = widgets.IntText(value=50,min=0,max=255,step=1.,description='',readout_format='d', continuous_update = False)
label_02 = widgets.Label(value='Threshold value')
l2 = widgets.jslink((threshold_01,'value'),(threshold_01_txt,'value'))

radius = widgets.IntSlider(value=10,min=1,max=10,step=1.,description='',readout_format='d', continuous_update = False)
radius_txt = widgets.IntText(value=10,min=1,max=10,step=1.,description='',readout_format='d', continuous_update = False)
label_03 = widgets.Label(value='Radius')
l3 = widgets.jslink((radius,'value'),(radius_txt,'value'))

threshold = widgets.IntSlider(value=50,min=0,max=255,step=1.,description='',readout_format='d', continuous_update = False)
threshold_txt = widgets.IntText(value=50,min=0,max=255,step=1.,description='',readout_format='d', continuous_update = False)
label_04 = widgets.Label(value='Threshold')
l4 = widgets.jslink((threshold,'value'),(threshold_txt,'value'))

dilation = widgets.IntSlider(value=0,min=0,max=15,step=1.,description='',readout_format='d', continuous_update = False)
dilation_txt = widgets.IntText(value=0,min=0,max=15,step=1.,description='',readout_format='d', continuous_update = False)
label_05 = widgets.Label(value='Dilation')
l5 = widgets.jslink((dilation,'value'),(dilation_txt,'value'))

rad  = widgets.FloatText(value=3,min=1,max=255,step=.1,description='Radius',readout_format='.1f', continuous_update = False)


refresh00 = widgets.Checkbox(value=True, description='refresh',disabled=False)
refresh10 = widgets.Checkbox(value=True, description='refresh',disabled=False)
refresh = widgets.Checkbox(value=True, description='refresh',disabled=False)
refresh01 = widgets.Checkbox(value=True, description='refresh',disabled=False)
refresh02 = widgets.Checkbox(value=True, description='refresh',disabled=False)
refresh03 = widgets.Checkbox(value=True, description='refresh',disabled=False)
refresh04 = widgets.Checkbox(value=True, description='refresh',disabled=False)
refresh05 = widgets.Checkbox(value=True, description='refresh',disabled=False)
refresh06 = widgets.Checkbox(value=True, description='Calculus',disabled=False)
refresh07 = widgets.Checkbox(value=False, description='Plot data',disabled=False)
refresh08 = widgets.Checkbox(value=True, description='Curve fitting',disabled=False)


weight_01 = widgets.FloatSlider(value=1,min=0,max=1,step=0.01,description='Weight',
                            readout_format='.1f',continuous_update=False)
weight_02 = widgets.FloatSlider(value=1,min=0,max=1,step=0.01,description='Weight',
                            readout_format='.1f',continuous_update=False)

retrieval_method = widgets.Dropdown(options=['cv2.RETR_TREE','cv2.RETR_EXTERNAL',
                    'cv2.RETR_LIST'],value='cv2.RETR_TREE', 
                    description='Retrieval method', disabled=False )

approximation_method = widgets.Dropdown(options=['cv2.CHAIN_APPROX_NONE','cv2.CHAIN_APPROX_SIMPLE',
                    'cv2.CHAIN_APPROX_TC89_L1','cv2.CHAIN_APPROX_TC89_KCOS'],
                    value='cv2.CHAIN_APPROX_NONE', description='Approximation method', disabled=False )




dilation02 = widgets.IntSlider(value=0,min=0,max=60,step=1.,description='Dilation',
                        readout_format='d', continuous_update = False)

kernel  = widgets.IntSlider(value=3,min=3,max=15,step=2,description='Kernel',
                          readout_format='d', continuous_update = False)


Smooth_filters = widgets.Dropdown(options=['Gaussian blur','Mean filter',
                    'Median filter', 'Bilateral filter','Original image'],value='Mean filter', 
                    description='Smooth filters', disabled=False )

sigmaX = widgets.FloatText(value=1,min=0,max=10,step=.1,description='Sigma X',
                       continuous_update=False, readout_format ='.1f')

sigmaY = widgets.FloatText(value=1,min=0,max=10,step=.1,description='Sigma Y',
                       continuous_update=False, readout_format ='.1f')

Update = widgets.Checkbox(value=False, description='Update',disabled=False)

Normalize = widgets.Checkbox(value=True, description='Normalize',disabled=False)

degree_01 = widgets.IntSlider(value=1,min=1,max=15,step=1,description='Drone',
                          readout_format='d', continuous_update = False)

degree_02 = widgets.IntSlider(value=1,min=1,max=15,step=1,description='Field',
                          readout_format='d', continuous_update = False)


save_01 = widgets.Checkbox(value=False, description='Save Image',disabled=False)
save_02 = widgets.Checkbox(value=False, description='Save Image',disabled=False)
save_03 = widgets.Checkbox(value=False, description='Save Image',disabled=False)
save_04 = widgets.Checkbox(value=False, description='Save Image',disabled=False)
save_05 = widgets.Checkbox(value=False, description='Save Image',disabled=False)
save_06 = widgets.Checkbox(value=False, description='Save Image',disabled=False)
save_07 = widgets.Checkbox(value=False, description='Save Image',disabled=False)
save_08 = widgets.Checkbox(value=False, description='Save Image',disabled=False)
save_09 = widgets.Checkbox(value=False, description='Save Image',disabled=False)
save_10 = widgets.Checkbox(value=False, description='Save Image',disabled=False)

address_01 = widgets.Text(value='Insert the addres to save the image',placeholder='Type something',description='',disabled=False)
address_02 = widgets.Text(value='Insert the addres to save the image',placeholder='Type something',description='',disabled=False)
address_03 = widgets.Text(value='Insert the addres to save the image',placeholder='Type something',description='',disabled=False)
address_04 = widgets.Text(value='Insert the addres to save the image',placeholder='Type something',description='',disabled=False)
address_05 = widgets.Text(value='Insert the addres to save the image',placeholder='Type something',description='',disabled=False)
address_06 = widgets.Text(value='Insert the addres to save the image',placeholder='Type something',description='',disabled=False)
address_07 = widgets.Text(value='Insert the addres to save the image',placeholder='Type something',description='',disabled=False)
address_08 = widgets.Text(value='Insert the addres to save the image',placeholder='Type something',description='',disabled=False)
address_09 = widgets.Text(value='Insert the addres to save the image',placeholder='Type something',description='',disabled=False)
address_10 = widgets.Text(value='Insert the addres to save the image',placeholder='Type something',description='',disabled=False)


dilation.layout.width = '80%'
dilation_txt.layout.width = '60%'
radius.layout.width = '50%'
radius_txt.layout.width = '40%'
min_distance.layout.width = '80%'
min_distance_txt.layout.width='60%'
threshold_01.layout.width = '80%'
threshold_01_txt.layout.width = '60%'
threshold.layout.width = '80%'
threshold_txt.layout.width = '60%'
kernel.layout.width = '97%'
sigmaX.layout.width = '80%'
sigmaY.layout.width = '80%'
address_01.layout.width='99%'
address_02.layout.width='99%'
address_06.layout.width='99%'
address_09.layout.width='99%'
address_10.layout.width='99%'
Smooth_filters.layout.width = '90%'

save_01.layout.margin = '0 0 0 -60px'
save_02.layout.margin = '0 0 0 -60px'
save_04.layout.margin = '0 0 0 -60px'
save_06.layout.margin = '0 0 0 -60px'
save_09.layout.margin = '0 0 0 -60px'
save_10.layout.margin = '0 0 0 -60px'
refresh00.layout.margin = '0 0 0 -60px'
refresh02.layout.margin = '0 0 0 -60px'
refresh05.layout.margin = '0 0 0 -110px'
refresh06.layout.margin = '0 0 0 -110px'
refresh07.layout.margin = '0 0 0 -110px'
min_distance.layout.margin = '0 0 0 40px'
min_distance_txt.layout.margin = '0 0 0 40px'
threshold_01.layout.margin = '0 0 0 40px'
threshold_01_txt.layout.margin = '0 0 0 40px'
threshold.layout.margin = '0 0 0 40px'
threshold_txt.layout.margin = '0 0 0 40px'
dilation.layout.margin = '0 0 0 40px'
dilation_txt.layout.margin = '0 0 0 40px'
Update.layout.margin = '0 0 0 -60px'
Normalize.layout.margin = '0 0 0 -70px'
min_distance.style.handle_color = 'orange'
threshold.style.handle_color = 'orange'
sigmaY.style.handle_color = 'orange'
sigmaX.style.handle_color = 'orange'

### <font color = gold> $ 2. \ \ \ \mathbb{Define \ the \ functions} $

In [ ]:
def smoothing_filters(img=fixed(img),Smooth_filters=Smooth_filters,
                      kernel=kernel,sigmaX=sigmaX,sigmaY=sigmaY,Update=False,Normalize=True,
                     save_01=save_01,address_01=address_01):
    global smooth_img
    if Update == True:
        if Smooth_filters == 'Gaussian blur':
            smooth_img = cv2.GaussianBlur(img,(kernel,kernel),sigmaX,sigmaY)
        elif Smooth_filters == 'Mean filter' and Normalize == True:
            smooth_img = cv2.boxFilter(img, 0, (kernel,kernel), 0, (-1,-1), True, cv2.BORDER_DEFAULT)
        elif Smooth_filters == 'Mean filter' and Normalize == False:
            smooth_img = cv2.boxFilter(img, 0, (kernel,kernel), 0, (-1,-1), False, cv2.BORDER_DEFAULT)
        elif Smooth_filters == 'Median filter':
            smooth_img = cv2.medianBlur(img,(kernel)) 
        elif Smooth_filters == 'Bilateral filter':
            smooth_img = cv2.bilateralFilter(img,kernel,75,75)
        else:
            smooth_img = img
            
        fig,ax = mpp.subplots(figsize=(5,2),dpi=300,facecolor='w')
        ax.set_title('Smoothed image')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(smooth_img,'Greens')
        if save_01 == True:
            fig.savefig(address_01)
            save_01 = False
        return smooth_img

#a = interactive(smoothing_filters,Smooth_filters=Smooth_filters,img=fixed(img),kernel_size=kernel_size,
#                sigmaX=sigmaX,sigmaY=sigmaY,Update=False)

a = widgets.interactive_output(smoothing_filters,{'Smooth_filters':Smooth_filters,'img':fixed(img),
                                  'kernel':kernel,'sigmaX':sigmaX,
                                  'sigmaY':sigmaY,'Update':Update,'Normalize':Normalize,
                                    'save_01':save_01,'address_01':address_01})

left_box_01 = VBox([Update,Smooth_filters])
left_box_01.layout.width = '35%'
left_box_01.layout.border = '4px solid cyan'
middle_box_01 = VBox([sigmaX,sigmaY])
middle_box_01.layout.width = '33%'
middle_box_01.layout.border = '4px solid cyan'
right_box_01 = VBox([kernel,Normalize,save_01,address_01])
right_box_01.layout.width = '40%'
right_box_01.layout.border = '4px solid cyan'
full_box_01 = HBox([left_box_01,middle_box_01,right_box_01])
full_box_01.layout.width = '100%'
full_box_01.layout.border = '4px solid royalblue'
#full_box.layout.width = '1000px'
#tab1 = VBox(children=a.children)
tab1 = VBox(children=[full_box_01,a])

In [ ]:
img = img.astype('uint8')
def local_max(img,min_distance=10,threshold_01=threshold_01,refresh00=True,save_02=save_02,address_02=address_02,
             min_distance_txt=min_distance_txt,threshold_01_txt=threshold_01_txt):
    global max_points
    if refresh00 ==True:
        fig,ax1 = mpp.subplots(figsize=(5,2),dpi=300,facecolor='w')
        ax1.set_title('Local maximas')
        ax1.set_xticks([])
        ax1.set_yticks([])
        ax1.imshow(smooth_img,'Greens')
        max_points = peak_local_max(smooth_img, min_distance,threshold_01)
        ax1.annotate("Number \nof points =",(130,100),color='k')
        ax1.text(350,100, len(max_points),color='k')
        ax1.plot(max_points[:, 1], max_points[:, 0], 'k.',markersize=1)
        if save_02 == True:
            fig.savefig(address_02)
            save_02 = False
        return max_points
        return img

p = widgets.interactive_output(local_max, {'img':fixed(img), 'min_distance':min_distance,
                'threshold_01':threshold_01,'refresh00':refresh00,'save_02':save_02,
                'address_02':address_02,'min_distance_txt':min_distance_txt,
                'threshold_01_txt':threshold_01_txt})

left_box_02 = VBox([refresh00,label_01,min_distance,min_distance_txt])
middle_box_02 = VBox([label_02,threshold_01,threshold_01_txt])
right_box_02 = VBox([save_02,address_02])
full_box_02 = HBox([left_box_02,middle_box_02,right_box_02])

left_box_02.layout.width = '33%'
middle_box_02.layout.width = '33%'
right_box_02.layout.width = '34%'
full_box_02.layout.width = '100%'
left_box_02.layout.border = '4px solid cyan'
middle_box_02.layout.border = '4px solid cyan'
right_box_02.layout.border = '4px solid cyan'
full_box_02.layout.border = '4px solid royalblue'
tab2 = VBox(children=[full_box_02,p])

In [ ]:
def draw_circles(radius=1,radius_txt=radius_txt,refresh=True,save_03=save_03,address_03=address_03):
    global img_points
    if refresh == True: 
        fig,ax2 = mpp.subplots(figsize=(5,2),dpi=300,facecolor='w')
        img_points = np.ones([img.shape[0],img.shape[1],3])
        img_points *= 255
        for i in range (max_points.shape[0]):
            cv2.circle(img_points,(max_points[i,1],max_points[i,0]),
                       radius=radius,color=(0,0,0),thickness=-1)
            i += 1
        ax2.set_title('Local maximas')
        ax2.set_xticks([])
        ax2.set_yticks([])
        ax2.imshow(img_points,'Greens')
        mpp.show()
        if save_03 == True:
            fig.savefig(address_03)
            save_03 = False
        return img_points
    
#out=widgets.Output(layout=Layout(height='300px', width = '400px', border='solid'))
q = widgets.interactive_output(draw_circles, {'radius':radius,'refresh':refresh,
                                'save_03':save_03,'address_03':address_03,'radius_txt':radius_txt})

inside_box = HBox([radius,radius_txt])
left_box_03 = VBox([label_03,refresh,inside_box])
right_box_03 = VBox([save_03,address_03])
full_box_03 = HBox([left_box_03,right_box_03])
left_box_03.layout.width = '50%'
right_box_03.layout.width = '50%'
full_box_03.layout.width = '100%'
left_box_03.layout.border = '4px solid cyan'
right_box_03.layout.border = '4px solid cyan'
full_box_03.layout.border = '4px solid royalblue'
tab3 = VBox(children=[full_box_03,q])

In [ ]:
def contours_labeled(retrieval_method = retrieval_method  ,
                     approximation_method = approximation_method,refresh01=True,
                    save_04=save_04,address_04=address_04):

    global img_centroids
    global img_points
    global contours01
    global circle_means
    if refresh01 == True:
        img_points_01 = np.copy(img_points)
        img_points_01 = img_points_01.astype('uint8')
        img_points_01 = cv2.cvtColor(img_points_01, cv2.COLOR_RGB2GRAY)
        img_centroids = np.zeros(img.shape, dtype=np.int32)
    
    
        if retrieval_method == 'cv2.RETR_EXTERNAL' and approximation_method == 'cv2.CHAIN_APPROX_NONE' :
            image,contours01,hierarchy = cv2.findContours(img_points_01,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
        elif retrieval_method == 'cv2.RETR_LIST' and approximation_method == 'cv2.CHAIN_APPROX_NONE' :
            image,contours01,hierarchy = cv2.findContours(img_points_01,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
        elif retrieval_method == 'cv2.RETR_TREE' and approximation_method == 'cv2.CHAIN_APPROX_NONE' :
            image,contours01,hierarchy = cv2.findContours(img_points_01,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
        elif retrieval_method == 'cv2.RETR_EXTERNAL' and approximation_method == 'cv2.CHAIN_APPROX_SIMPLE' :
            image,contours01,hierarchy = cv2.findContours(img_points_01,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        elif retrieval_method == 'cv2.RETR_LIST' and approximation_method == 'cv2.CHAIN_APPROX_SIMPLE' :
            image,contours01,hierarchy = cv2.findContours(img_points_01,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        elif retrieval_method == 'cv2.RETR_TREE' and approximation_method == 'cv2.CHAIN_APPROX_SIMPLE' :
            image,contours01,hierarchy = cv2.findContours(img_points_01,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        elif retrieval_method == 'cv2.RETR_EXTERNAL' and approximation_method == 'cv2.CHAIN_APPROX_TC89_L1' :
            image,contours01,hierarchy = cv2.findContours(img_points_01,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_TC89_L1)
        elif retrieval_method == 'cv2.RETR_LIST' and approximation_method == 'cv2.CHAIN_APPROX_TC89_L1' :
            image,contours01,hierarchy = cv2.findContours(img_points_01,cv2.RETR_LIST,cv2.CHAIN_APPROX_TC89_L1)
        elif retrieval_method == 'cv2.RETR_TREE' and approximation_method == 'cv2.CHAIN_APPROX_TC89_L1' :
            image,contours01,hierarchy = cv2.findContours(img_points_01,cv2.RETR_TREE,cv2.CHAIN_APPROX_TC89_L1)
        elif retrieval_method == 'cv2.RETR_EXTERNAL' and approximation_method == 'cv2.CHAIN_APPROX_TC89_KCOS' :
            image,contours01,hierarchy = cv2.findContours(img_points_01,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_TC89_KCOS)
        elif retrieval_method == 'cv2.RETR_LIST' and approximation_method == 'cv2.CHAIN_APPROX_TC89_KCOS' :
            image,contours01,hierarchy = cv2.findContours(img_points_01,cv2.RETR_LIST,cv2.CHAIN_APPROX_TC89_KCOS)
        else:
            image,contours01,hierarchy = cv2.findContours(img_points_01,cv2.RETR_TREE,cv2.CHAIN_APPROX_TC89_KCOS)

        for i in range(len(contours01)):
            cv2.drawContours(image=img_centroids,contours=contours01,contourIdx=i,
                         color=(i,i,i),thickness=cv2.FILLED) 
        
        circle_means = np.empty([len(contours01)],dtype='float32')
        for i in range(len(contours01)):
            a = (img_centroids==i)* img
            circle_means[i] = (stats.tmean((a),(0,300),(False,True)))
            
        #circle_means_c = circle_means - circle_means.min(axis=0,keepdims=True)
        #circle_means_c /= circle_means.max(axis=0, keepdims=True)
        #circle_means_c += 0.1
        
        fig,ax3 = mpp.subplots(figsize=(5,2),dpi=300,facecolor='w')   
        ax3.set_title('Contours labeled')
        ax3.set_xticks([])
        ax3.set_yticks([])
        ax3.annotate("Number \nof points =",(130,100),color='k')
        ax3.text(320,100, len(contours01),color='k')
        ax3.imshow(img_centroids,'Greens')
        mpp.show()
        if save_04 == True:
            fig.savefig(address_04)
            save_04 = False
        return img_centroids,contours01,circle_means

r = widgets.interactive_output(contours_labeled,{'retrieval_method':retrieval_method,
                'approximation_method':approximation_method,'refresh01':refresh01,
               'save_04':save_04,'address_04':address_04})

left_box_04 = VBox([refresh01,retrieval_method,approximation_method])
right_box_04 = VBox([save_04,address_04])
full_box_04 = HBox([left_box_04,right_box_04])
left_box_04.layout.width = '50%'
right_box_04.layout.width = '50%'
full_box_04.layout.width = '100%'
left_box_04.layout.border = '4px solid cyan'
right_box_04.layout.border = '4px solid cyan'
full_box_04.layout.border = '4px solid royalblue'
tab4 = VBox(children=[full_box_04,r])

In [ ]:
def redraw_circles(rad=15,refresh10=True,save_05=save_05,address_05=address_05):
    global circle_means
    global circle_means_01
    global img_points_02
    global centers01

    if refresh10 == True:
        fig,ax4 = mpp.subplots(figsize=(5,2),dpi=300,facecolor='w')
        img_points_02 = np.ones([img.shape[0],img.shape[1],3],dtype='uint8')
        img_points_02 *= 255
        centers = [None]*len(contours01)
        radius = [None]*len(contours01)
        
        for i in range(len(contours01)):
            centers[i],radius[i] = cv2.minEnclosingCircle(contours01[i])
            
        centers01 = np.asarray(centers)
        centers01 = centers01.astype('float32')
        # Formula of the second order regression !!!
        circle_means_01 = (0.4119*circle_means*circle_means - 29.68*circle_means - 35.2)/rad
        circle_means_01 = circle_means_01.astype('uint8')
        
        for i in range(len(contours01)):
            cv2.circle(img_points_02,(centers01[i][0],centers01[i][1]),
                       radius=circle_means_01[i], color=(0,0,0), thickness=-1)
            
        img_points_02 = img_points_02[:,:,0]
        ax4.set_title('Local maximas')
        ax4.set_xticks([])
        ax4.set_yticks([])
        ax4.imshow(img_points_02,'Greys')
        mpp.show()
        if save_05 == True:
            fig.savefig(address_05)
            save_05 = False
        return img_points_02,centers01
    
#out=widgets.Output(layout=Layout(height='300px', width = '400px', border='solid'))
z = widgets.interactive_output(redraw_circles,{'rad':rad, 'refresh10':refresh10,'save_05':save_05,
                                'address_05':address_05})

left_box_05 = VBox([refresh10,rad])
right_box_05 = VBox([save_05,address_05])
full_box_05 = HBox([left_box_05,right_box_05])
left_box_05.layout.width = '50%'
right_box_05.layout.width = '50%'
full_box_05.layout.width = '100%'
left_box_05.layout.border = '4px solid cyan'
right_box_05.layout.border = '4px solid cyan'
full_box_05.layout.border = '4px solid royalblue'
tab5 = VBox(children=[full_box_05,z])

In [ ]:
def unknown_area(img,refresh02=True,dilation=16,threshold=0,save_06=save_06,address_06=address_06,
                dilation_txt=dilation_txt,threshold_txt=threshold_txt):
    global img_points_02
    global centroids_bw
    global unknown
    if refresh02 == True:
        thresh_value , binary_image = cv2.threshold(img,threshold,255,cv2.THRESH_BINARY)
        kernel = np.ones((6,6),np.uint8)
        binary_image = cv2.dilate(binary_image,kernel,iterations=dilation)
        img_points_02 = np.uint8(img_points_02)
        ret , centroids_bw = cv2.threshold(img_points_02,0,255,cv2.THRESH_BINARY_INV)
        unknown = cv2.subtract(binary_image,centroids_bw)
    
        fig,ax5 = mpp.subplots(figsize=(5,2),dpi=300,facecolor='w')   
        ax5.set_title('Unknown area')
        ax5.set_xticks([])
        ax5.set_yticks([])
        ax5.imshow(unknown,'Greens')
        mpp.show()
        if save_06 == True:
            fig.savefig(address_06)
            save_06 = False
        return centroids_bw,unknown

s = widgets.interactive_output(unknown_area,{'img':fixed(img),'refresh02':refresh02,'dilation':dilation,
                                'threshold':threshold,'save_06':save_06,'address_06':address_06,
                                'dilation_txt':dilation_txt,'threshold_txt':threshold_txt})


left_box_06 = VBox([refresh02,label_05,dilation,dilation_txt])
middle_box_06 = VBox([label_04,threshold,threshold_txt])
right_box_06 = VBox([save_06,address_06])
full_box_06 = HBox([left_box_06,middle_box_06,right_box_06])
left_box_06.layout.width = '33%'
middle_box_06.layout.width = '33%'
right_box_06.layout.width = '34%'
full_box_06.layout.width = '100%'
left_box_06.layout.border = '4px solid cyan'
right_box_06.layout.border = '4px solid cyan'
full_box_06.layout.border = '4px solid royalblue'
tab6 = VBox(children=[full_box_06,s])

In [ ]:
def connect_component(refresh03=True,save_07=save_07,address_07=address_07):
    global unknown
    global centroids_bw
    global conec_comp
    if refresh03 == True:
        centroids_bw = np.uint8(centroids_bw)
        stats,conec_comp = cv2.connectedComponents(centroids_bw)
        conec_comp += 1 
        conec_comp[unknown==255] = 0
        fig,ax5 = mpp.subplots(figsize=(5,2),dpi=300,facecolor='w')   
        ax5.set_title('Connect Components')
        ax5.set_xticks([])
        ax5.set_yticks([])
        ax5.imshow(conec_comp,'Greens')
        mpp.show()
        if save_07 == True:
            fig.savefig(address_07)
            save_07 = False
        return conec_comp
    
t = widgets.interactive_output(connect_component,{'refresh03':refresh03,'save_07':save_07,
                                'address_07':address_07})

left_box_07 = VBox([refresh03])
right_box_07 = VBox([save_07,address_07])
full_box_07 = HBox([left_box_07,right_box_07])
left_box_07.layout.width = '50%'
right_box_07.layout.width = '50%'
full_box_07.layout.width = '100%'
left_box_07.layout.border = '4px solid cyan'
right_box_07.layout.border = '4px solid cyan'
full_box_07.layout.border = '4px solid royalblue'
tab7 = VBox(children=[full_box_07,t])

In [ ]:
def watershed_segmentation(refresh04=True,save_08=save_08,address_08=address_08):
    global watershed
    global gray_image
    global centers01
    global img
    if refresh04==True:
        gray_image = np.empty([img.shape[0],img.shape[1], 3], dtype=np.uint8)
        watershed = np.copy(conec_comp)
        watershed = watershed.astype('int32')
        watershed = cv2.watershed(gray_image, watershed);
        gray_image[watershed == -1] = [255,255,255]
        gray_image = cv2.cvtColor(gray_image,cv2.COLOR_RGB2GRAY)
        binary_image = np.copy(img)
        for x in range(img.shape[0]):
            for y in range(img.shape[1]):
                if binary_image[x,y]>0:
                    binary_image[x,y]=1
                else:
                    binary_image[x,y]=0    
        gray_image = binary_image*gray_image
        fig,ax6 = mpp.subplots(figsize=(5,2),dpi=300,facecolor='w')   
        ax6.annotate("Number of \n segmentations =",(100,130),color='k')
        ax6.text(470,100,np.amax(watershed),color='k')
        ax6.set_title('Watershed Segmentation')
        ax6.set_xticks([])
        ax6.set_yticks([])
        ax6.imshow(watershed,'Greens')
        ax6.scatter(centers01[:,0],centers01[:,1],color='k',s=1)
        mpp.show()
        if save_08 == True:
            fig.savefig(address_08)
            save_08 = False
        return watershed,gray_image
u = widgets.interactive_output(watershed_segmentation, {'refresh04':refresh04,
                                'save_08':save_08,'address_08':address_08})

left_box_08 = VBox([refresh04])
right_box_08 = VBox([save_08,address_08])
full_box_08 = HBox([left_box_08,right_box_08])
left_box_08.layout.width = '50%'
right_box_08.layout.width = '50%'
full_box_08.layout.width = '100%'
left_box_08.layout.border = '4px solid cyan'
right_box_08.layout.border = '4px solid cyan'
full_box_08.layout.border = '4px solid royalblue'
tab8 = VBox(children=[full_box_08,u])

In [ ]:
def dam_division_lines(dilation02=0,refresh05=True,weight_01=weight_01,weight_02=weight_02,
                       refresh06 = refresh06,save_09=save_09,address_09=address_09):
    global gray_image
    global dam_lines
    global rgb_image
    global img
    global crown_area
    global max_height
    global watershed
    global heights
    global df_whole
    global df_whole_slice
    global centers01
    
    
    if refresh05 == True:
        dam_lines = np.copy(gray_image)
        kernel = np.ones((3,3),np.uint8)
        dam_lines = cv2.dilate(dam_lines,kernel,iterations=dilation02)
        dam_lines = cv2.merge((dam_lines,dam_lines,dam_lines))
        img01 = cv2.merge((img,img,img))
        result_01 = cv2.addWeighted(dam_lines,weight_01,rgb_image,weight_02,0)
        result_02 = cv2.addWeighted(dam_lines,weight_01,img01,weight_02,0)
        fig,(ax7,ax8) = mpp.subplots(ncols=1,nrows=2,figsize=(5,5),dpi=900,facecolor='w')   
        ax7.set_title('RGB and Dam lines')
        ax7.set_xticks([])
        ax7.set_yticks([])
        ax7.imshow(result_01)
        ax7.scatter(centers01[:,0],centers01[:,1],color='r',s=1)
        ax8.set_title('DEM and Dam lines')
        ax8.set_xticks([])
        ax8.set_yticks([])
        ax8.imshow(result_02)
        ax8.scatter(centers01[:,0],centers01[:,1],color='r',s=1)
        mpp.show()
        if save_09 == True:
            fig.savefig(address_09)
            save_09 = False
        if refresh06 == True:
            segmentations = np.zeros([np.amax(watershed),watershed.shape[0],watershed.shape[1]],dtype='uint8')
            heights = np.zeros([np.amax(watershed),watershed.shape[0],watershed.shape[1]],dtype='float32')
            mean_height = np.zeros([np.amax(watershed)],dtype='float32')
            max_height = np.zeros([np.amax(watershed)],dtype='float32')
            crown_area = np.zeros([np.amax(watershed)],dtype='float32')
            for i in range(2, np.amax(watershed)+1, 1):
                segmentations[i-2] = (watershed==i)
                heights[i-2] = segmentations[i-2]*height_plantio
                mean_height[i-2] = stats.tmean((heights[i-2]),(0,300),(False,True))
                max_height[i-2] = np.max(heights[i-2])
                image,contours,hierarchy = cv2.findContours(segmentations[i-2],cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
                crown_area[i-2] = cv2.contourArea(contours[0])
            crown_area *= 10.4
            max_height *= 100 
            mean_height *=100
            heights *=100
        
        # Field measurement threshold
        #minval = np.amin(heights[np.nonzero(heights)])
        #df_whole = df_whole[df_whole.iloc[:,1] >= minval]
        
            minval = np.amin(max_height[np.nonzero(max_height)])
            df_whole_slice = df_whole[df_whole.iloc[:,1] >= minval]
        
        # Drone values threshold
            minval_01 = np.amin(max_height[np.nonzero(max_height)])
            max_height = max_height[max_height >= minval_01]
            minval_02 = np.amin(crown_area[np.nonzero(crown_area)])
            crown_area = crown_area[crown_area >= minval_02]
        return dam_lines,crown_area,max_height,heights,df_whole_slice,result_01,centers01,result_02
    else:
        pass

v = widgets.interactive_output(dam_division_lines,{'dilation02':dilation02,'refresh05':refresh05,
                'weight_01':weight_01, 'weight_02':weight_02,'refresh06':refresh06,
               'save_09':save_09,'address_09':address_09})

left_box_09 = VBox([refresh05,refresh06])
middle_box_09 = VBox([dilation02,weight_01,weight_02])
right_box_09 = VBox([save_09,address_09])
full_box_09 = HBox([left_box_09,middle_box_09,right_box_09])
left_box_09.layout.width = '25%'
middle_box_09.layout.width = '41%'
right_box_09.layout.width = '34%'
full_box_09.layout.width = '100%'
left_box_09.layout.border = '4px solid cyan'
middle_box_09.layout.border = '4px solid cyan'
right_box_09.layout.border = '4px solid cyan'
full_box_09.layout.border = '4px solid royalblue'
tab9 = VBox(children=[full_box_09,v])

In [ ]:
def regression_analysis(refresh07=refresh07,degree_01=degree_01,degree_02=degree_02,save_10=save_10,address_10=address_10):
    # Read the raster DEM in float64 format
    global crown_area
    global max_height
    global heights
    global df_whole
    global df_whole_slice
    global reg_pol_02
    if refresh07 == True:
        fig,ax9 = mpp.subplots(dpi=300,figsize=(5,3),facecolor='w')
        
        # Limits selection
        xx_min = np.amin([np.amin(max_height),np.amin(df_whole_slice.iloc[:,1])])
        xx_max = np.amax([np.amax(max_height),np.amax(df_whole_slice.iloc[:,1])])
        yy_min = np.amin([np.amin(crown_area),np.amin(df_whole_slice.iloc[:,2])])
        yy_max = np.amax([np.amax(crown_area),np.amax(df_whole_slice.iloc[:,2])])
        
        xp_01 = np.linspace(np.amin(df_whole.iloc[:,1]),xx_max)
        xp = np.linspace(xx_min,xx_max,100)
        mpp.axvline(xx_min,ls='--',lw=1,color='k')
        mpp.axvline(xx_max,ls='--',lw=1,color='k')
        
        # Plot the data from drone
        ax9.scatter(max_height,crown_area,color='gold',alpha=0.5,s=5)
        # Adjust the curve from drone data
        reg_pol_01 = np.poly1d(np.polyfit(max_height,crown_area,degree_01))
        ax9.plot(xp,reg_pol_01(xp),color='gold')
        r_square_01 = r2_score(crown_area,reg_pol_01(max_height))

         # Plot the  data from field measurement
        ax9.scatter(df_whole.iloc[:,1],df_whole.iloc[:,2],color='red',alpha=0.5,s=5)
        # Adjust the curve from field data
        reg_pol_02 = np.poly1d(np.polyfit(df_whole.iloc[:,1],df_whole.iloc[:,2],degree_02))
        ax9.plot(xp_01,reg_pol_02(xp_01),color='red')
        r_square_02 = r2_score(df_whole.iloc[:,2],reg_pol_02(df_whole.iloc[:,1]))
        
        # Correlation between field data and drone regressions.
        corr = pearsonr(reg_pol_01(xp),reg_pol_02(xp))
        
        rmse_01 = sqrt(mean_squared_error(crown_area,reg_pol_01(max_height)));
        rmse_02 = sqrt(mean_squared_error(df_whole.iloc[:,2],reg_pol_02(df_whole.iloc[:,1])));
        mae_01 = mean_absolute_error(crown_area,reg_pol_01(max_height))
        mae_02 = mean_absolute_error(df_whole.iloc[:,2],reg_pol_02(df_whole.iloc[:,1]))
        
        ax9.annotate('$\mathrm{R^2 \ field \ =}$  ',xy=(0.03,0.93) ,color='black',xycoords='axes fraction')
        ax9.text(0.18,0.963,r_square_02.round(2),color='k',verticalalignment='top',transform=ax9.transAxes)
        
        ax9.annotate('$\mathrm{R^2 \ drone \ =}$  ',xy=(0.03,0.83) ,color='black',xycoords='axes fraction')
        ax9.text(0.19,0.863,r_square_01.round(2),color='k',verticalalignment='top',transform=ax9.transAxes)
        
        ax9.annotate('$\mathrm{Correlation \ =}$  ',xy=(0.03,0.73) ,color='black',xycoords='axes fraction')
        ax9.text(0.21,0.763,corr[0].round(2),color='k',verticalalignment='top',transform=ax9.transAxes)
        
        ax9.annotate('$\mathrm{Field \ count =} $  ',xy=(0.03,0.63) ,color='black',xycoords='axes fraction')
        ax9.text(0.2,0.663,len(df_whole_slice.index),color='k',verticalalignment='top',transform=ax9.transAxes)
        
        ax9.annotate('$\mathrm{Drone \ count =} $  ',xy=(0.03,0.53) ,color='black',xycoords='axes fraction')
        ax9.text(0.22,0.563,len(max_height),color='k',verticalalignment='top',transform=ax9.transAxes)
        
        #ax9.annotate(str(reg_pol_02[2].round(4),xy=(0.03,0.43) ,color='black',xycoords='axes fraction')
        ax9.text(0.03,0.48,'$\mathrm{Field \ Regression :}$',color='k',verticalalignment='top',transform=ax9.transAxes)
        ax9.text(0.03,0.40,reg_pol_02[2].round(4),color='k',verticalalignment='top',transform=ax9.transAxes)
        ax9.text(0.105,0.405,'$\mathrm{* x^2}$',color='k',verticalalignment='top',transform=ax9.transAxes)
        ax9.text(0.15,0.40,'$\mathrm{-}$',color='k',verticalalignment='top',transform=ax9.transAxes)
        ax9.text(0.18,0.40,np.abs(reg_pol_02[1]).round(4),color='k',verticalalignment='top',transform=ax9.transAxes)
        ax9.text(0.265,0.40,'$\mathrm{* x}$',color='k',verticalalignment='top',transform=ax9.transAxes)
        ax9.text(0.30,0.40,'$\mathrm{+}$',color='k',verticalalignment='top',transform=ax9.transAxes)
        ax9.text(0.33,0.40,int(reg_pol_02[0]),color='k',verticalalignment='top',transform=ax9.transAxes)
        
        ax9.text(0.03,0.33,'$\mathrm{Drone \ Regression :}$',color='k',verticalalignment='top',transform=ax9.transAxes)
        ax9.text(0.03,0.25,reg_pol_01[2].round(4),color='k',verticalalignment='top',transform=ax9.transAxes)
        ax9.text(0.105,0.255,'$\mathrm{* x^2}$',color='k',verticalalignment='top',transform=ax9.transAxes)
        ax9.text(0.15,0.25,'$\mathrm{-}$',color='k',verticalalignment='top',transform=ax9.transAxes)
        ax9.text(0.18,0.25,np.abs(reg_pol_01[1]).round(2),color='k',verticalalignment='top',transform=ax9.transAxes)
        ax9.text(0.25,0.25,'$\mathrm{* x}$',color='k',verticalalignment='top',transform=ax9.transAxes)
        ax9.text(0.285,0.25,'$\mathrm{+}$',color='k',verticalalignment='top',transform=ax9.transAxes)
        ax9.text(0.315,0.25,int(reg_pol_01[0]),color='k',verticalalignment='top',transform=ax9.transAxes)
        
        #(str(reg_pol_02[2].round(4)) + '*' + 'x²' + ' ' + str(reg_pol_02[1].round(4)) +
         #      '*' + 'x' + ' + ' + str(reg_pol_02[0].round(4)))
        
       # ax9.annotate('$\mathrm{RMSE \ field =} $  ',xy=(0.03,0.43) ,color='black',xycoords='axes fraction')
       #ax9.text(0.15,0.463,"%.2f" % rmse_01,color='k',verticalalignment='top',transform=ax9.transAxes)
        
       # ax9.annotate('$\mathrm{RMSE \ drone =} $  ',xy=(0.03,0.33) ,color='black',xycoords='axes fraction')
       # ax9.text(0.17,0.363,"%.2f" % rmse_02,color='k',verticalalignment='top',transform=ax9.transAxes)
        
       # ax9.annotate('$\mathrm{MAE \ field =} $  ',xy=(0.03,0.23) ,color='black',xycoords='axes fraction')
       # ax9.text(0.14,0.263,"%.2f" % mae_01,color='k',verticalalignment='top',transform=ax9.transAxes)
        
       # ax9.annotate('$\mathrm{MAE \ drone =} $  ',xy=(0.03,0.13) ,color='black',xycoords='axes fraction')
       # ax9.text(0.15,0.163,"%.2f" % mae_02,color='k',verticalalignment='top',transform=ax9.transAxes)
        
        
        
        drone_patch = Line2D([],[],color='gold',linestyle='-',linewidth=2,label='$\mathrm{Drone}$')
        field_patch = Line2D([],[],color='red',linestyle='-',linewidth=2,label='$\mathrm{Field}$')
        leg1 = mpp.legend(handles=[field_patch,drone_patch],loc='upper center')
        
        mpp.title('$\mathrm{Plantio \ of \ block \ 01}$')
        ax9.set_xticks([xx_min,xx_max])
        ax9.set_yticks([yy_min,yy_max])
        ax9.set_xlabel('$\mathrm{Height \ (cm)}$',labelpad=-5,horizontalalignment='center',
                      position=(0.7,0))
        ax9.set_ylabel('$\mathrm{Crown \ Area \ (cm^2)}$',labelpad=-15)
        mpp.tight_layout()
        if save_10 == True:
            fig.savefig(address_10)
            save_10 = False
        return reg_pol_02

x = widgets.interactive_output(regression_analysis,{'refresh07':refresh07,'degree_01':degree_01,
                                     'degree_02':degree_02,'save_10':save_10,
                                     'address_10':address_10})

left_box_10 = VBox([refresh07])
middle_box_10 = VBox([degree_01,degree_02])
right_box_10 = VBox([save_10,address_10])
full_box_10 = HBox([left_box_10,middle_box_10,right_box_10])
left_box_10.layout.width = '25%'
middle_box_10.layout.width = '41%'
right_box_10.layout.width = '34%'
full_box_10.layout.width = '100%'
left_box_10.layout.border = '4px solid cyan'
middle_box_10.layout.border = '4px solid cyan'
right_box_10.layout.border = '4px solid cyan'
full_box_10.layout.border = '4px solid royalblue'
tab10 = VBox(children=[full_box_10,x])

In [ ]:
tab1 = VBox(children=[full_box_01,a])
tab2 = VBox(children=[full_box_02,p])
tab3 = VBox(children=[full_box_03,q])
tab4 = VBox(children=[full_box_04,r])
tab5 = VBox(children=[full_box_05,z])
tab6 = VBox(children=[full_box_06,s])
tab7 = VBox(children=[full_box_07,t])
tab8 = VBox(children=[full_box_08,u])
tab9 = VBox(children=[full_box_09,v])
tab10 = VBox(children=[full_box_10,x])

tab = widgets.Tab(children=[tab1,tab2,tab3,tab4,tab5,tab6,tab7,tab8,tab9,tab10])
tab.set_title(0,'Smooth filters')
tab.set_title(1,'Local maximas and Draw circles')
tab.set_title(2,'Draw Circles')
tab.set_title(3,'Contours labeled')
tab.set_title(4,'Redraw Circles')
tab.set_title(5,'Unknown area')
tab.set_title(6,'Connect Components')
tab.set_title(7,'Watershed segmentation')
tab.set_title(8,'Dam lines')
tab.set_title(9,'Regression Analysis')

In [ ]:
display(tab)

In [ ]:
for i in range(len(contours01)):
    cv2.circle(dam_lines,(centers01[i][0],centers01[i][1]),radius=10, color=(255,215,0), thickness=-1)
    
HV = cv2.cvtColor(dam_lines,cv2.COLOR_RGB2BGR)

In [ ]:
t = raw_input("What address save the image ? suggestions : .jpg")
cv2.imwrite(t,HV)